In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("results_full.csv")
df = df[df["split_method"] != "random_split"]

In [3]:
rename_map = {
    'ConstantPredictor': 'Const.',
    'FTTransformer': 'FT-Trans.',
    'LGBMRegressor': 'GBT',
    'LGBMClassifier': 'GBT',
    'LinearRegressor': 'Lin. Regr.',
    'RandomForestRegressor': 'RF',
    'RandomForestClassifier': 'RF',
    'ResNet': 'ResNet',
    'TabPFNRegressor': 'TabPFN',
    'TabPFNClassifier': 'TabPFN',
    'LogisticRegressor': 'Log. Regr.',
    'GPBoost_LogLoss': 'GP',
    'GPBoost_Accuracy': 'GP',
    'GPBoost_CRPS': 'GP',
    'GPBoost_RMSE': 'GP'

}


Average Difference and Average Relative Difference RMSE

In [4]:
metric = "RMSE"
dfm   = df[df["metric"] == metric]

pivot_rmse = dfm.pivot_table(
    index="task_id",
    columns="model",
    values="value",
    aggfunc="mean"
)
pivot_rmse = pivot_rmse.rename(columns=rename_map)

pivot_rmse.to_csv(f"avg_{metric}_per_task_per_model.csv", float_format="%.3f")


In [5]:
def avg_relative_diff(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:

    if error_metric:
        best_per_task = pivot.min(axis=1)   # best is minimum
        # (v - best)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)
                         .div(best_per_task, axis=0)
                         * 100)
    else:
        best_per_task = pivot.max(axis=1)   # best is maximum
        # (best - v)/best * 100
        rel_diff = (best_per_task.sub(pivot, axis=0)
                              .div(best_per_task, axis=0)
                              * 100)

    # average across tasks (skipping any NaNs)
    return rel_diff.mean(axis=0)


avg_diff_rmse = avg_relative_diff(pivot_rmse, error_metric=True)

print("Average relative difference (RMSE):")
print(avg_diff_rmse.round(2))


Average relative difference (RMSE):
model
Const.           312.38
Engression       609.34
FT-Trans.         33.58
GP               232.59
GBT               69.86
Lin. Regr.    107722.44
MLP              129.14
RF                73.09
ResNet           112.93
TabPFN            32.76
dtype: float64


In [6]:
def normalized_accuracy(pivot: pd.DataFrame,
                        error_metric: bool = True) -> pd.Series:

    if error_metric:
        best = pivot.min(axis=1)  

        mid  = pivot.apply(
            lambda s: s.dropna().nlargest(3).min(),
            axis=1
        )
        norm = (
            pivot
            .rsub(mid,       axis=0)    # mid[t] - pivot.loc[t, m]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )
    else:
        best = pivot.max(axis=1)
        mid  = pivot.apply(
            lambda s: s.dropna().nsmallest(3).max(),
            axis=1
        )
        # 3) norm_acc per cell = (acc - mid) / (best - mid), clipped
        norm = (
            pivot
            .rsub(mid,          axis=0)   # pivot.loc[t,m] - mid[t]
            .mul(-1)
            .div(mid - best, axis=0)
            .clip(0, 1)
        )

    return norm.mean(axis=0)


avg_norm_acc_rmse = normalized_accuracy(pivot_rmse, error_metric=True)
print("Average normalized accuracy (from RMSE):")
print((100 * avg_norm_acc_rmse).round(2))


Average normalized accuracy (from RMSE):
model
Const.        10.82
Engression    34.87
FT-Trans.     63.48
GP            30.91
GBT           75.55
Lin. Regr.     7.57
MLP           56.41
RF            68.84
ResNet        44.95
TabPFN        89.67
dtype: float64


In [7]:
def avg_rank(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:
  
    ranks = pivot.rank(
        axis=1,
        method="average",   
        ascending=error_metric
    )

    return ranks.mean(axis=0)

avg_rank_rmse = avg_rank(pivot_rmse, error_metric=True)
print("Average rank (RMSE):")
print(avg_rank_rmse.round(2))



Average rank (RMSE):
model
Const.        8.56
Engression    6.33
FT-Trans.     4.09
GP            6.67
GBT           3.42
Lin. Regr.    8.36
MLP           4.94
RF            4.31
ResNet        5.83
TabPFN        2.00
dtype: float64


In [8]:
avg_diff    = avg_relative_diff(pivot_rmse, error_metric=True)           # in %
avg_acc     = normalized_accuracy(pivot_rmse, error_metric=True) * 100   # convert to % 
avg_rank    = avg_rank(pivot_rmse, error_metric=True)

metrics = pd.DataFrame(
    [avg_diff, avg_acc, avg_rank],
    index=["Avg. diff.", "Avg. acc.", "Avg. rank."]
)

pivot_with_summaries = pd.concat([pivot_rmse, metrics])

out = pivot_with_summaries.reset_index().rename(columns={"index":"task_id"})

out.to_csv(f"avg_{metric}_with_summary.csv", float_format="%.3f", index=False)

latex = out.to_latex(index=False, escape=True, float_format="%.3f")
with open(f"{metric}_results.tex","w") as f:
    f.write(latex)

Average Difference and Average Relative Difference CRPS

In [9]:
metric1 = "CRPS"
dfm1   = df[df["metric"] == metric1]

pivot_crps = dfm1.pivot_table(
    index="task_id",
    columns="model",
    values="value",
    aggfunc="mean"
)

pivot_crps = pivot_crps.rename(columns=rename_map)
pivot_crps.to_csv(f"avg_{metric1}_per_task_per_model.csv", float_format="%.3f")


In [10]:
def avg_relative_diff(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:

    if error_metric:
        best_per_task = pivot.min(axis=1)   # best is minimum
        # (v - best)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)
                         .div(best_per_task, axis=0)
                         * 100)
    else:
        best_per_task = pivot.max(axis=1)   # best is maximum
        # (best - v)/best * 100
        rel_diff = (best_per_task.sub(pivot, axis=0)
                              .div(best_per_task, axis=0)
                              * 100)

    # average across tasks (skipping any NaNs)
    return rel_diff.mean(axis=0)


avg_diff_crps = avg_relative_diff(pivot_crps, error_metric=True)

print("Average relative difference (CRPS):")
print(avg_diff_crps.round(2))


Average relative difference (CRPS):
model
Const.          552.40
DGBT          69849.59
DRF              87.31
Engression      582.52
FT-Trans.        42.25
GP              178.99
GBT              67.05
Lin. Regr.    73945.52
MLP             102.55
RF               79.32
ResNet          136.75
TabPFN           26.05
dtype: float64


In [11]:
def normalized_accuracy(pivot: pd.DataFrame,
                        error_metric: bool = True) -> pd.Series:

    if error_metric:
        best = pivot.min(axis=1)  

        mid  = pivot.apply(
            lambda s: s.dropna().nlargest(3).min(),
            axis=1
        )
        norm = (
            pivot
            .rsub(mid,       axis=0)    # mid[t] - pivot.loc[t, m]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )
    else:
        best = pivot.max(axis=1)
        mid  = pivot.apply(
            lambda s: s.dropna().nsmallest(3).max(),
            axis=1
        )
        # 3) norm_acc per cell = (acc - mid) / (best - mid), clipped
        norm = (
            pivot
            .rsub(mid,          axis=0)   # pivot.loc[t,m] - mid[t]
            .mul(-1)
            .div(mid - best, axis=0)
            .clip(0, 1)
        )

    return norm.mean(axis=0)


avg_norm_acc_crps = normalized_accuracy(pivot_crps, error_metric=True)
print("Average normalized accuracy (from CRPS):")
print((100 * avg_norm_acc_crps).round(2))


Average normalized accuracy (from CRPS):
model
Const.         5.91
DGBT          69.05
DRF           74.47
Engression    16.80
FT-Trans.     65.82
GP            39.27
GBT           73.10
Lin. Regr.    11.82
MLP           60.05
RF            68.13
ResNet        51.21
TabPFN        84.15
dtype: float64


In [12]:
def avg_rank(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:
  
    ranks = pivot.rank(
        axis=1,
        method="average",   
        ascending=error_metric
    )

    return ranks.mean(axis=0)

avg_rank_crps = avg_rank(pivot_crps, error_metric=True)
print("Average rank (CRPS):")
print(avg_rank_crps.round(2))



Average rank (CRPS):
model
Const.        10.78
DGBT           4.22
DRF            4.50
Engression     9.81
FT-Trans.      4.94
GP             8.14
GBT            4.33
Lin. Regr.     9.75
MLP            6.00
RF             5.44
ResNet         6.81
TabPFN         2.69
dtype: float64


In [13]:
avg_diff1    = avg_relative_diff(pivot_crps, error_metric=True)           # in %
avg_acc1     = normalized_accuracy(pivot_crps, error_metric=True) * 100   # convert to % 
avg_rank1    = avg_rank(pivot_crps, error_metric=True)

metrics = pd.DataFrame(
    [avg_diff1, avg_acc1, avg_rank1],
    index=["Avg. diff.", "Avg. acc.", "Avg. rank."]
)

pivot_with_summaries1 = pd.concat([pivot_crps, metrics])

out1 = pivot_with_summaries1.reset_index().rename(columns={"index":"task_id"})

out1.to_csv(f"avg_{metric1}_with_summary.csv", float_format="%.3f", index=False)

latex1 = out1.to_latex(index=False, escape=True, float_format="%.3f")
with open(f"{metric1}_results.tex","w") as f:
    f.write(latex1)

Average Difference and Average Relative Difference LOGLOSS

In [14]:
metric2 = "LogLoss"
dfm2   = df[df["metric"] == metric2]

pivot_ll = dfm2.pivot_table(
    index="task_id",
    columns="model",
    values="value",
    aggfunc="mean"
)
pivot_ll = pivot_ll.rename(columns=rename_map)
pivot_ll.to_csv(f"avg_{metric2}_per_task_per_model.csv", float_format="%.3f")


In [15]:
def avg_relative_diff(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:

    if error_metric:
        best_per_task = pivot.min(axis=1)   # best is minimum
        # (v - best)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)
                         .div(best_per_task, axis=0)
                         * 100)
    else:
        best_per_task = pivot.max(axis=1)   # best is maximum
        # (best - v)/best * 100
        rel_diff = (best_per_task.sub(pivot, axis=0)
                              .div(best_per_task, axis=0)
                              * 100)

    return rel_diff.mean(axis=0)


avg_diff_ll = avg_relative_diff(pivot_ll, error_metric=True)

print("Average relative difference (LogLoss):")
print(avg_diff_ll.round(2))


Average relative difference (LogLoss):
model
Const.        254.35
Engression    178.82
FT-Trans.      18.93
GP             52.44
GBT             7.77
Log. Regr.    130.82
MLP            24.26
RF             20.11
ResNet         30.15
TabPFN          3.39
dtype: float64


In [16]:
def normalized_accuracy(pivot: pd.DataFrame,
                        error_metric: bool = True) -> pd.Series:

    if error_metric:
        best = pivot.min(axis=1)  

        mid  = pivot.apply(
            lambda s: s.dropna().nlargest(3).min(),
            axis=1
        )
        norm = (
            pivot
            .rsub(mid,       axis=0)    # mid[t] - pivot.loc[t, m]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )
    else:
        best = pivot.max(axis=1)
        mid  = pivot.apply(
            lambda s: s.dropna().nsmallest(3).max(),
            axis=1
        )
        # 3) norm_acc per cell = (acc - mid) / (best - mid), clipped
        norm = (
            pivot
            .rsub(mid,          axis=0)   # pivot.loc[t,m] - mid[t]
            .mul(-1)
            .div(mid - best, axis=0)
            .clip(0, 1)
        )

    return norm.mean(axis=0)


avg_norm_acc_ll = normalized_accuracy(pivot_ll, error_metric=True)
print("Average normalized accuracy (from LogLoss):")
print((100 * avg_norm_acc_ll).round(2))


Average normalized accuracy (from LogLoss):
model
Const.         6.26
Engression     7.62
FT-Trans.     53.70
GP            50.33
GBT           87.26
Log. Regr.     8.57
MLP           44.76
RF            81.32
ResNet        37.71
TabPFN        98.23
dtype: float64


In [17]:
def avg_rank(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:
  
    ranks = pivot.rank(
        axis=1,
        method="average",   
        ascending=error_metric
    )

    return ranks.mean(axis=0)

avg_rank_ll = avg_rank(pivot_ll, error_metric=True)
print("Average rank (LogLoss):")
print(avg_rank_ll.round(5))



Average rank (LogLoss):
model
Const.        9.00000
Engression    8.26087
FT-Trans.     5.27273
GP            5.43478
GBT           2.69565
Log. Regr.    7.91304
MLP           5.26087
RF            3.08696
ResNet        6.17391
TabPFN        1.69565
dtype: float64


In [18]:
avg_diff2    = avg_relative_diff(pivot_ll, error_metric=True)           # in %
avg_acc2     = normalized_accuracy(pivot_ll, error_metric=True) * 100   # convert to % 
avg_rank2    = avg_rank(pivot_ll, error_metric=True)

metrics = pd.DataFrame(
    [avg_diff2, avg_acc2, avg_rank2],
    index=["Avg. diff.", "Avg. acc.", "Avg. rank."]
)

pivot_with_summaries2 = pd.concat([pivot_ll, metrics])

out2 = pivot_with_summaries2.reset_index().rename(columns={"index":"task_id"})

out2.to_csv(f"avg_{metric2}_with_summary.csv", float_format="%.6f", index=False)

latex2 = out2.to_latex(index=False, escape=True, float_format="%.6f")
with open(f"{metric2}_results.tex","w") as f:
    f.write(latex2)

Average Difference and Average Relative Difference Accuracy

In [19]:
metric3 = "Accuracy"
dfm3   = df[df["metric"] == metric3]

pivot_acc = dfm3.pivot_table(
    index="task_id",
    columns="model",
    values="value",
    aggfunc="mean"
)

pivot_acc = pivot_acc.rename(columns=rename_map)
pivot_acc.to_csv(f"avg_{metric3}_per_task_per_model.csv", float_format="%.5f")


In [20]:
def avg_relative_diff(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:

    if error_metric:
        best_per_task = pivot.min(axis=1)   # best is minimum
        # (v - best)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)
                         .div(best_per_task, axis=0)
                         * 100)
    else:
        best_per_task = pivot.max(axis=1)   # best is maximum
        # (best - v)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)     
                         .mul(-1)                         
                         .div(best_per_task , axis=0)
                         * 100)

    return rel_diff.mean(axis=0)


avg_diff_accuracy = avg_relative_diff(pivot_acc, error_metric=False)

print("\nAverage relative difference (Accuracy):")
print(avg_diff_accuracy.round(2))



Average relative difference (Accuracy):
model
Const.        45.87
Engression    33.49
FT-Trans.      3.85
GP             6.44
GBT            1.40
Log. Regr.     9.07
MLP            4.10
RF             2.13
ResNet         3.92
TabPFN         0.28
dtype: float64


In [21]:
def normalized_accuracy(pivot: pd.DataFrame,
                        error_metric: bool = True) -> pd.Series:

    if error_metric:
        best = pivot.min(axis=1)  

        mid  = pivot.apply(
            lambda s: s.dropna().nlargest(3).min(),
            axis=1
        )
        norm = (
            pivot
            .rsub(mid,       axis=0)    # mid[t] - pivot.loc[t, m]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )
    else:
        best = pivot.max(axis=1)
        mid  = pivot.apply(
            lambda s: s.dropna().nsmallest(3).max(),
            axis=1
        )
        # 3) norm_acc per cell = (acc - mid) / (best - mid), clipped
        norm = (
            pivot
            .rsub(mid,          axis=0)   # pivot.loc[t,m] - mid[t]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )

    return norm.mean(axis=0)


avg_norm_acc_acc = normalized_accuracy(pivot_acc, error_metric=False)
print("Average normalized accuracy (from Accuracy):")
print((100 * avg_norm_acc_acc).round(3))


Average normalized accuracy (from Accuracy):
model
Const.         0.000
Engression     0.000
FT-Trans.     56.745
GP            34.940
GBT           83.850
Log. Regr.    18.139
MLP           54.470
RF            79.462
ResNet        58.280
TabPFN        96.424
dtype: float64


In [22]:
def avg_rank(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:
  
    ranks = pivot.rank(
        axis=1,
        method="average",   
        ascending=error_metric
    )

    return ranks.mean(axis=0)

avg_rank_acc = avg_rank(pivot_acc, error_metric=False)
print("Average rank (Accuracy):")
print(avg_rank_acc.round(2))



Average rank (Accuracy):
model
Const.        9.87
Engression    9.00
FT-Trans.     4.68
GP            6.61
GBT           3.00
Log. Regr.    7.13
MLP           4.70
RF            3.26
ResNet        4.78
TabPFN        1.74
dtype: float64


In [23]:
avg_diff3    = avg_relative_diff(pivot_acc, error_metric=False)           # in %
avg_acc3     = normalized_accuracy(pivot_acc, error_metric=False) * 100   # convert to % 
avg_rank3    = avg_rank(pivot_acc, error_metric=False)

metrics = pd.DataFrame(
    [avg_diff3, avg_acc3, avg_rank3],
    index=["Avg. diff.", "Avg. acc.", "Avg. rank."]
)

pivot_with_summaries3 = pd.concat([pivot_acc, metrics])

out3 = pivot_with_summaries3.reset_index().rename(columns={"index":"task_id"})

out3.to_csv(f"avg_{metric3}_with_summary.csv", float_format="%.3f", index=False)

latex3 = out3.to_latex(index=False, escape=True, float_format="%.3f")
with open(f"{metric3}_results.tex","w") as f:
    f.write(latex3)